# 🤖 Ensemble Learning - Stacking (Stacked Generalization) with Wine Quality Dataset
### Predicting Wine Quality Using Multiple Base Models & Meta-Model


In [1]:
# 📌 Step 1: Import Required Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import StackingClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, accuracy_score

import warnings
warnings.filterwarnings('ignore')


In [2]:
# 📌 Step 2: Load and Explore the Wine Quality Dataset
df = pd.read_csv("winequality-red.csv", sep=';')
df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
# 📌 Step 3: Check Class Distribution (Quality)

df['quality'].value_counts()


5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
# 📌 Step 4: Convert to Binary Classification Problem
# Let's assume: quality >= 6 is 'good' (1), else 'not good' (0)
df['quality_binary'] = df['quality'].apply(lambda x: 1 if x >= 6 else 0)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_binary
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [6]:
# 📌 Step 5: Split Dataset into Features and Target
X = df.drop(['quality', 'quality_binary'], axis=1)

y = df['quality_binary'] # target variable


In [7]:
# 📌 Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
X_train.shape, y_train.shape

((1279, 11), (1279,))

In [9]:
X_test.shape, y_test.shape

((320, 11), (320,))

In [10]:
# 📌 Step 7: Feature Scaling (Important for KNN & SVM)

# Scale features using StandardScaler for better performance of KNN & SVM

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
X_train_scaled

array([[-0.59096646, -0.08898536, -0.47926523, ...,  1.3506617 ,
         0.43263109, -0.8741677 ],
       [-0.93251464,  0.07896192, -0.73734044, ...,  1.93541674,
        -0.11163301, -0.5942794 ],
       [ 0.43367806, -0.14496778,  0.14011526, ...,  0.05120607,
         0.31168351,  0.05879332],
       ...,
       [-0.64789116,  0.75075102, -1.25349085, ...,  1.41563448,
        -0.53494953, -0.6875755 ],
       [-1.27406281,  1.00267194, -0.89218556, ...,  1.87044395,
         0.00931457, -0.22109499],
       [ 0.83215092, -0.08898536, -0.42765019, ..., -0.53354896,
        -0.47447574,  0.05879332]])

In [12]:
# 📌 Step 7: Feature Scaling (Important for KNN & SVM)
# Scale features using StandardScaler for better performance of KNN & SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# 📌 Step 8: Define Base Models (Level-0 Learners)
# Define the base learners: Decision Tree, KNN, and SVM with probability output

base_learners = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('svm', SVC(probability=True, random_state=42))
]


In [19]:
# 📌 Step 9: Define Meta Model (Level-1 Learner)

meta_model = LogisticRegression()

# Define the meta-model that combines base models' predictions: Logistic Regression

In [20]:
# 📌 Step 10: Initialize Stacking Classifier
stack_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_model,
    cv=5
)


In [21]:
# 📌 Step 11: Train the Stacking Model
stack_model.fit(X_train_scaled, y_train)


StackingClassifier(cv=5,
                   estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                               ('knn', KNeighborsClassifier()),
                               ('svm', SVC(probability=True, random_state=42))],
                   final_estimator=LogisticRegression())

In [23]:
# 📌 Step 12: Predict and Evaluate the Model
y_pred = stack_model.predict(X_test_scaled)

print("📊 Classification Report:\n")
print(classification_report(y_test, y_pred))

print("✅ Accuracy Score:", accuracy_score(y_test, y_pred))


📊 Classification Report:

              precision    recall  f1-score   support

           0       0.73      0.77      0.75       149
           1       0.79      0.75      0.77       171

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320

✅ Accuracy Score: 0.75625


We trained 3 different models (Decision Tree, KNN, SVM) as base models.

Their individual predictions were used to train a final Logistic Regression model.

Stacking performed well, and accuracy improved by combining strengths of all models.

You can also compare this result with AdaBoost and Gradient Boosting to highlight performance differences.